In [ ]:
#Third Model: Mild Dementia vs Signs of very mild Dementia
#getting the data for the first model ready using DataLoader
#add a few images for moderate but changing brightness 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])
dataset = datasets.ImageFolder(root="Processed_MRI_Data_For_Third_Model", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
print(dataset.classes)

['Signs_of_Mild_Dimentia', 'Signs_of_Very_Mild_Dementia']


In [ ]:
#Bring in Resnet pretrained model
import torch
import torchvision.models as models
res_net_model = models.resnet50(pretrained=True)

C:\Users\holde\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\holde\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#create the training set and validation set, but for each batch, weight the images so that
#we get close to an equal number of images of each class in each batch.
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter

train_indices, val_indices = train_test_split(np.arange(len(dataset)), test_size=0.25, stratify=dataset.targets)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

#get the count of each class
class_counts = [0,0]
for class_i in train_indices:
    c_i = dataset.targets[class_i]
    class_counts[c_i] += 1
class_proportions = [1/class_counts[0] , 1/class_counts[1]]
class_proportions = [prop * 100000 for prop in class_proportions]
sample_weights = []
for idx in train_indices:  # Iterate through the training indices
    label = dataset.targets[idx]
    sample_weights.append(class_proportions[0] if label == 0 else class_proportions[1])
sampler = WeightedRandomSampler(sample_weights, num_samples=len(train_dataset), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=50, sampler=sampler)
#print(sample_weights)
eval_loader = DataLoader(val_dataset, batch_size = 50, shuffle = False)

num_batches_to_check = 10  # Number of batches you want to inspect


for i, (images, labels) in enumerate(train_loader):
    if i >= num_batches_to_check:
        break
    label_count = Counter(labels.tolist())  # Count the labels in the batch
    print(f"Batch {i + 1} label count:", label_count)
print("-----------------------------")
for i, (images, labels) in enumerate(eval_loader):
    if i >= num_batches_to_check:
        break
    label_count = Counter(labels.tolist())  # Count the labels in the batch
    print(f"Batch {i + 1} label count:", label_count)

Batch 1 label count: Counter({1: 27, 0: 23})
Batch 2 label count: Counter({0: 32, 1: 18})
Batch 3 label count: Counter({0: 29, 1: 21})
Batch 4 label count: Counter({0: 25, 1: 25})
Batch 5 label count: Counter({1: 25, 0: 25})
Batch 6 label count: Counter({1: 25, 0: 25})
Batch 7 label count: Counter({0: 32, 1: 18})
Batch 8 label count: Counter({1: 28, 0: 22})
Batch 9 label count: Counter({1: 25, 0: 25})
Batch 10 label count: Counter({1: 27, 0: 23})
-----------------------------
Batch 1 label count: Counter({1: 42, 0: 8})
Batch 2 label count: Counter({1: 36, 0: 14})
Batch 3 label count: Counter({1: 35, 0: 15})
Batch 4 label count: Counter({1: 37, 0: 13})
Batch 5 label count: Counter({1: 33, 0: 17})
Batch 6 label count: Counter({1: 35, 0: 15})
Batch 7 label count: Counter({1: 38, 0: 12})
Batch 8 label count: Counter({1: 35, 0: 15})
Batch 9 label count: Counter({1: 35, 0: 15})
Batch 10 label count: Counter({1: 40, 0: 10})


In [ ]:
#add output layer
import torch
import torchvision.models as models

res_net_model.fc = torch.nn.Linear(res_net_model.fc.in_features, 2)

In [ ]:
#Define the loss function and the optimizer

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(res_net_model.parameters(), lr=0.000001)


In [ ]:
'''
import torch
import torch.nn as nn
import torchvision.models as models
num_ftrs = res_net_model.fc.in_features
res_net_model.fc = nn.Sequential(
    nn.Dropout(p=0.4),  # 50% Dropout to prevent overfitting
    nn.Linear(num_ftrs, 2)  # Assuming 2 classes in your dataset
)

#begin forward and backwards propogation
from collections import Counter
num_epochs = 1
for epoch in range(num_epochs):
    res_net_model.train()
    running_loss = 0.0
    #iterate through a batch
    print(len(train_loader))
    for ind, (inputs, labels) in enumerate(train_loader):
        #print(ind)
        optimizer.zero_grad() #zero out gradients
        batch_outputs = res_net_model(inputs)
        loss = criterion(batch_outputs, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model weights

        running_loss += loss.item()
        if (ind + 1) % 10 == 0:
            label_count = Counter(labels.tolist())  # Count the labels in the batch
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{ind+1}/{len(train_loader)}], Loss: {loss.item():.4f} label count: {label_count}")
        if((ind + 1) % 150 == 0 or (ind+1) % len(train_loader)==0):
            torch.save(res_net_model.state_dict(), f"Third_Model_checkpoint_step_{ind+1}.pth")
            print("Model weights saved")            

    #start evaluation
    res_net_model.eval()
    correct, total = 0 , 0
    with torch.no_grad():
        for inputs, labels in eval_loader:
            eval_outputs = res_net_model(inputs)
            _, predicted = torch.max(eval_outputs, 1)  # Get predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Evaluation Accuracy: {accuracy:.2f}%")
            
torch.save(res_net_model.state_dict(), "progress_model_weights.pth")
'''

'\nimport torch\nimport torch.nn as nn\nimport torchvision.models as models\nnum_ftrs = res_net_model.fc.in_features\nres_net_model.fc = nn.Sequential(\n    nn.Dropout(p=0.4),  # 50% Dropout to prevent overfitting\n    nn.Linear(num_ftrs, 2)  # Assuming 2 classes in your dataset\n)\n\n#begin forward and backwards propogation\nfrom collections import Counter\nnum_epochs = 1\nfor epoch in range(num_epochs):\n    res_net_model.train()\n    running_loss = 0.0\n    #iterate through a batch\n    print(len(train_loader))\n    for ind, (inputs, labels) in enumerate(train_loader):\n        #print(ind)\n        optimizer.zero_grad() #zero out gradients\n        batch_outputs = res_net_model(inputs)\n        loss = criterion(batch_outputs, labels)\n        loss.backward()  # Backpropagation\n        optimizer.step()  # Update model weights\n\n        running_loss += loss.item()\n        if (ind + 1) % 10 == 0:\n            label_count = Counter(labels.tolist())  # Count the labels in the batch\n 

In [ ]:
#training algorithm I used while observing. Load in the weights previously gotten during the training. 
#train it, observe the loss each batch, every two batches, you can be asked to save the current weights.
#This allows us to view the loss closely, and I believe this allows us to reach the global minimum faster.
#add a drop out layer to prevent overfitting due to our unequal class sizes.
import torch
import torch.nn as nn
import torchvision.models as models
num_ftrs = res_net_model.fc.in_features
res_net_model.fc = nn.Sequential(
    nn.Dropout(p=0.4),  # 50% Dropout to prevent overfitting
    nn.Linear(num_ftrs, 2)  # Assuming 2 classes in your dataset
)
res_net_model.load_state_dict(torch.load("Third_Model_checkpoint_step_281.pth"))
from collections import Counter
num_epochs = 1
for epoch in range(num_epochs):
    res_net_model.train()
    running_loss = 0.0
    #iterate through a batch
    print(len(train_loader))
    for ind, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad() #zero out gradients
        batch_outputs = res_net_model(inputs)
        loss = criterion(batch_outputs, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model weights

        running_loss += loss.item()
        label_count = Counter(labels.tolist())  # Count the labels in the batch
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{ind+1}/{len(train_loader)}], Loss: {loss.item():.5f} label count: {label_count}")
        if (ind + 1) % 20 == 0:
            save = input("Save this model?")
            if save == "y":
                torch.save(res_net_model.state_dict(), f"NEWLY_SAVED_MODEL_3_Updated_checkpoint_step_{ind+1}.pth")
                print("Model weights saved")            
            

            
torch.save(res_net_model.state_dict(), "progress_model_weights.pth")

In [ ]:
#evaluate the model weights
res_net_model.load_state_dict(torch.load("NEWLY_SAVED_MODEL_3_Updated_checkpoint_step_280.pth"))
res_net_model.eval()
correct, total = 0 , 0
with torch.no_grad():
    for ind, (inputs, labels) in enumerate(eval_loader):
        eval_outputs = res_net_model(inputs)
        _, predicted = torch.max(eval_outputs, 1)  # Get predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f"Batch {ind+1}: Accuracy: {correct / total} , {correct}: {total}")

accuracy = 100 * correct / total
print(f"Evaluation Accuracy: {accuracy:.2f}%")